In [ ]:
#install talos and opencv-python if not done before

!pip install -q --user talos
!pip install opencv-python

#install tensorflow-gpu version 1.12.0. Make sure you run Python 3.6 with Cuda 9.0

!pip install tensorflow-gpu == 1.12.0

In [2]:
#import modules

import os
import numpy as np
import scipy
import cv2
from PIL import Image
import itertools
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Conv2D, Dropout, BatchNormalization, GlobalAveragePooling2D, MaxPooling2D, Flatten, Activation
from keras.activations import softmax
from keras.losses import categorical_crossentropy
from keras.utils import np_utils
from keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam
import talos as ta
from talos.model.normalizers import lr_normalizer

In [9]:
#set paths to images

infected_path = 'C:/Users/mikes/Desktop/milou/Natural Computing/cell_images/Parasitized/' 
uninfected_path = 'C:/Users/mikes/Desktop/milou/Natural Computing/cell_images/Uninfected/'

In [10]:
#load and preprocess (i.e., resize) images. Code adapted from the following kaggle kernel: 
#https://www.kaggle.com/kushal1996/detecting-malaria-cnn

infected = os.listdir(infected_path) 
uninfected = os.listdir(uninfected_path)

data = []
labels = []
    
for i in infected:
    try:

        image = cv2.imread(infected_path+i)
        image_array = Image.fromarray(image , 'RGB')
        resize_img = image_array.resize((100 , 100))
        data.append(np.array(resize_img))
        labels.append(1)
            
    except AttributeError:
        print('')
    
for u in uninfected:
    try:

        image = cv2.imread(uninfected_path+u)
        image_array = Image.fromarray(image , 'RGB')
        resize_img = image_array.resize((100 , 100))
        data.append(np.array(resize_img))
        labels.append(0)

    except AttributeError:
        print('')
    
cells = np.array(data)
labels = np.array(labels)

np.save('Cells' , cells)
np.save('Labels' , labels)
    
n = np.arange(cells.shape[0])
np.random.shuffle(n)
cells = cells[n]
labels = labels[n]

nb_classes = 2 

labels = np_utils.to_categorical(labels, nb_classes)

input_shape = cells.shape[1:]
   
cells = cells.astype('float32')
cells /= 255

In [11]:
#set channel, batch_size and epochs parameters

channel = 3 
batch_size = 32
epochs = 3

In [12]:
#declare the base architecture and parameters to optimize in the custom model. Code adapted from the github page:
#https://github.com/sivaramakrishnan-rajaraman/
#Deep-Neural-Ensembles-toward-Malaria-Parasite-Detection-in-Thin-Blood-Smear-Images/blob/master/custom_cnn_optimization.ipynb

def custom_model_fn(X_train, Y_train, X_test, Y_test, params):
    conv_dropout = float(params['conv_dropout'])
    dense1_neuron = int(params['dense1_neuron'])
    model = Sequential()
    model.add(BatchNormalization(input_shape=X_train.shape[1:]))
    model.add(Conv2D(64, (5, 5), padding='same', activation=params['activation']))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(Dropout(conv_dropout))

    model.add(BatchNormalization(input_shape=X_train.shape[1:]))
    model.add(Conv2D(128, (5, 5), padding='same', activation=params['activation']))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(conv_dropout))

    model.add(BatchNormalization(input_shape=X_train.shape[1:]))
    model.add(Conv2D(256, (5, 5), padding='same', activation=params['activation']))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(Dropout(conv_dropout))

    model.add(GlobalAveragePooling2D())
    model.add(Dense(dense1_neuron))
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout']))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.compile(optimizer=params['optimizer'](), loss='categorical_crossentropy',metrics=['accuracy'])
    
    out = model.fit(
        X_train, Y_train, epochs=epochs, batch_size=batch_size, 
        verbose=1,
        validation_data=[X_test, Y_test]
    )
    
    return out, model

In [13]:
#declare the parameters to optimize. Code adapted from the github page:
#https://github.com/sivaramakrishnan-rajaraman/
#Deep-Neural-Ensembles-toward-Malaria-Parasite-Detection-in-Thin-Blood-Smear-Images/blob/master/custom_cnn_optimization.ipynb

para = {
    'dense1_neuron': [256, 512],
    'activation': ['relu', 'elu', 'tanh', 'sigmoid', 'hard_sigmoid','softplus','linear'],
    'conv_dropout': [0.25, 0.5],
    'optimizer':  [Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam],
    'dropout': [0.25, 0.5]
}

In [14]:
#run the experiment with talosCode adapted from the github page:
#https://github.com/sivaramakrishnan-rajaraman/
#Deep-Neural-Ensembles-toward-Malaria-Parasite-Detection-in-Thin-Blood-Smear-Images/blob/master/custom_cnn_optimization.ipynb

scan_results = ta.Scan(cells, labels, para, custom_model_fn)

  0%|          | 0/392 [00:00<?, ?it/s]

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 19290 samples, validate on 8268 samples
Epoch 1/3
   32/19290 [..............................] - ETA: 1:14:35 - loss: 1.1686 - acc: 0.5000

KeyboardInterrupt: 

In [ ]:
#get the model ID that gives the best values for validation accuracy. Code copied from the github page:
#https://github.com/sivaramakrishnan-rajaraman/
#Deep-Neural-Ensembles-toward-Malaria-Parasite-Detection-in-Thin-Blood-Smear-Images/blob/master/custom_cnn_optimization.ipynb

model_id = scan_results.data['val_acc'].astype('float').argmax() - 1
model_id + 1

In [ ]:
#load the best model. Code copied from the github page:
#https://github.com/sivaramakrishnan-rajaraman/
#Deep-Neural-Ensembles-toward-Malaria-Parasite-Detection-in-Thin-Blood-Smear-Images/blob/master/custom_cnn_optimization.ipynb

model = model_from_json(scan_results.saved_models[model_id])
model.set_weights(scan_results.saved_weights[model_id])
model.summary()

In [ ]:
#get the validation accuracy of the best model. Code copied from the github page:
#https://github.com/sivaramakrishnan-rajaraman/
#Deep-Neural-Ensembles-toward-Malaria-Parasite-Detection-in-Thin-Blood-Smear-Images/blob/master/custom_cnn_optimization.ipynb

report.high('val_acc')

In [ ]:
#save the model. Code copied from the github page:
#https://github.com/sivaramakrishnan-rajaraman/
#Deep-Neural-Ensembles-toward-Malaria-Parasite-Detection-in-Thin-Blood-Smear-Images/blob/master/custom_cnn_optimization.ipynb

model.save('best_model.h5')